In [1]:
import numpy as np, pandas as pd
from datetime import datetime

In [2]:
start_date = datetime(2021, 1, 1).date()
end_date = datetime(2023, 12, 31).date()

In [3]:
def standardize_datetime_one_arg(df, col, date_format): 
    def date_fn(d, date_format): 
        if type(d) == float or d == 'nan':
            return datetime.strptime('2000-01-01', '%Y-%m-%d').date() # will get filtered out
        return datetime.strptime(d, date_format).date()

    df['std_date'] = df[col].apply(date_fn, args=(date_format,))
    df = df[(df['std_date'] >= start_date) & (df['std_date'] <= end_date)]
    return df
                                   
def standardize_datetime_two_args(df, col1, date_format1, col2, date_format2):
    def date_fn(row, col1, date_format1, col2, date_format2):
        d1 = str(row[col1])
        d2 = str(row[col2])
        if type(d1) != float and d1 != 'nan':
            return datetime.strptime(d1, date_format1).date()
        elif type(d2) != float and d2 != 'nan': 
            return datetime.strptime(d2, date_format2).date()        
        else: 
            return datetime.strptime('2000-01-01', '%Y-%m-%d').date() # will get filtered out
    
    df['std_date'] = df.apply(date_fn, args=(col1, date_format1, col2, date_format2), axis=1)
    df = df[(df['std_date'] >= start_date) & (df['std_date'] <= end_date)]
    return df

In [67]:
def standardize_with_mapping_new(df, city_name, file_name, mapping_columns):
    mapping = pd.read_csv('mappings/mapping_' + file_name + '.csv', index_col=None, dtype=str)
    df['join_key'] = ''
    mapping['join_key'] = ''
#     missing_mappings=[]
    for key in mapping_columns: 
        if key in mapping_columns:
            df['join_key'] = df['join_key'] + df[key].fillna('').astype(str)
            mapping['join_key'] = mapping['join_key'] + mapping[key].fillna('').astype(str)
       
    ### DROPOUT ###
    dropout = pd.merge(df, mapping, how='left', on='join_key')
    print("Dropout:", dropout[dropout['category_1'].astype(str) == 'nan'].shape)
    ###############

    df = pd.merge(df, mapping, how='inner', on='join_key')
    df['city'] = city_name
    df = df[['std_date', 'city', 'category_1', 'category_2', 'category_3']]
    df.to_csv('standardized/standardized_incidents_' + file_name + '.csv', index=False)
    print('Length: ', len(df))
    return df

In [5]:
def standardize_with_mapping(df, city_name, file_name, mapping_columns):
    mapping = pd.read_csv('mappings/mapping_' + file_name + '.csv', index_col=None, dtype=str)
    df['join_key'] = ''
    mapping['join_key'] = ''
    for key in mapping_columns: 
        df['join_key'] = df['join_key'] + df[key].fillna('').astype(str)
        mapping['join_key'] = mapping['join_key'] + mapping[key].fillna('').astype(str)

    ### DROPOUT ###
    dropout = pd.merge(df, mapping, how='left', on='join_key')
    print("Dropout:", dropout[dropout['category_1'].astype(str) == 'nan'].shape)
    ###############

    df = pd.merge(df, mapping, how='inner', on='join_key')
    df['city'] = city_name
    df = df[['std_date', 'city', 'category_1', 'category_2', 'category_3']]
    df.to_csv('standardized/standardized_incidents_' + file_name + '.csv', index=False)
    print('Length: ', len(df))
    return df

# Atlanta

In [78]:
city_name = 'Atlanta'
file_name = 'atlanta'
mapping_columns = ['NIBRS Code Name']

date_col1 = 'Report Date'
date_format1 = '%m/%d/%Y %I:%M:%S %p'
date_col2 = 'Offense Start Date'
date_format2 = '%m/%d/%Y %I:%M:%S %p'

In [80]:
# Read raw
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)

# Datetime
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)

std = standardize_with_mapping_new(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (16025, 34)
Length:  104852


,std_date,city,category_1,category_2,category_3
0,2021-12-31,Atlanta,Drug,Drug,Drug
1,2021-06-24,Atlanta,Drug,Drug,Drug
2,2021-04-07,Atlanta,Drug,Drug,Drug
3,2021-01-01,Atlanta,Drug,Drug,Drug
4,2021-01-01,Atlanta,Drug,Drug,Drug


# Austin

In [9]:
city_name = 'Austin'
file_name = 'austin'
mapping_columns = ['crime_type', 'ucr_code']

date_col1 = 'occ_date'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [10]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (83139, 34)
Length:  174495


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Austin,Violent,Assault,Simple Assault
1,2021-01-01,Austin,Violent,Assault,Simple Assault
2,2021-01-01,Austin,Violent,Assault,Simple Assault
3,2021-01-01,Austin,Violent,Assault,Simple Assault
4,2021-01-01,Austin,Violent,Assault,Simple Assault


# Baltimore

In [11]:
city_name = 'Baltimore'
file_name = 'baltimore'
mapping_columns = ['Description']

date_col1 = 'CrimeDateTime'
date_format1 = '%Y/%m/%d %H:%M:%S+%f'
# date_col2 = 
# date_format2 = 

In [12]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std['CrimeDateTime'] = pd.to_datetime(std['CrimeDateTime'], unit='ms')

# Format the datetime column
std['CrimeDateTime'] = std['CrimeDateTime'].dt.strftime('%Y/%m/%d %H:%M:%S+%f')
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (148, 27)
Length:  1852


,std_date,city,category_1,category_2,category_3
0,2023-12-16,Baltimore,Violent,Assault,Aggravated Assault
1,2023-12-19,Baltimore,Violent,Assault,Aggravated Assault
2,2023-12-19,Baltimore,Violent,Assault,Aggravated Assault
3,2023-12-19,Baltimore,Violent,Assault,Aggravated Assault
4,2023-12-15,Baltimore,Violent,Assault,Aggravated Assault


# Boston

In [13]:
city_name = 'Boston'
file_name = 'boston'
mapping_columns = ['OFFENSE_DESCRIPTION']

date_col1 = 'OCCURRED_ON_DATE'
date_format1 = '%Y-%m-%d %H:%M:%S+%f'
# date_col2 = 
# date_format2 = 

In [14]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
def convert_to_datetime_with_offset(date_string):
    try:
        return pd.to_datetime(date_string, format='%Y-%m-%d %H:%M:%S+%f')
    except ValueError:
        return pd.to_datetime(date_string, format='%Y-%m-%d %H:%M:%S')

# Apply the function to your 'OCCURRED_ON_DATE' column
std['OCCURRED_ON_DATE'] = std['OCCURRED_ON_DATE'].apply(convert_to_datetime_with_offset)
# std['OCCURRED_ON_DATE'] = pd.to_datetime(std['OCCURRED_ON_DATE'], format='%Y-%m-%d %H:%M:%S+%f')
std['OCCURRED_ON_DATE'] = std['OCCURRED_ON_DATE'].dt.strftime('%Y-%m-%d %H:%M:%S+%f')

std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (138906, 23)
Length:  82556


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Boston,Property,Theft,Other Theft
1,2021-01-01,Boston,Property,Theft,Other Theft
2,2021-01-01,Boston,Property,Theft,Other Theft
3,2021-01-01,Boston,Property,Theft,Other Theft
4,2021-01-01,Boston,Property,Theft,Other Theft


# Chicago

In [15]:
city_name = 'Chicago'
file_name = 'chicago'
mapping_columns = ['iucr']

date_col1 = 'date'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [16]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)

std = standardize_datetime_one_arg(std, date_col1, date_format1)

std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (91004, 28)
Length:  631353


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Chicago,Property,White Collar,Fraud
1,2021-01-01,Chicago,Property,White Collar,Fraud
2,2021-01-01,Chicago,Property,White Collar,Fraud
3,2021-01-01,Chicago,Property,White Collar,Fraud
4,2021-01-01,Chicago,Property,White Collar,Fraud


# Cincinnati

In [17]:
city_name = 'Cincinnati'
file_name = 'cincinnati'
mapping_columns = ['ucr_group', 'offense']

date_col1 = 'date_from'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
date_col2 = 'date_reported'
date_format2 = '%Y-%m-%dT%H:%M:%S.%f'

In [18]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (19297, 47)
Length:  82732


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Cincinnati,Property,Theft,Unclassified
1,2021-01-01,Cincinnati,Property,Theft,Unclassified
2,2021-01-01,Cincinnati,Property,Theft,Unclassified
3,2021-01-01,Cincinnati,Property,Theft,Unclassified
4,2021-01-01,Cincinnati,Property,Theft,Unclassified


# Dallas

In [19]:
city_name = 'Dallas'
file_name = 'dallas'
mapping_columns = ['ucr_offense', 'nibrs_crime', 'nibrs_crime_category']

date_col1 = 'date1'
date_format1 = '%Y-%m-%d %H:%M:%S.0%f'
# date_col2 = 
# date_format2 = 

In [20]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (130379, 94)
Length:  291492


,std_date,city,category_1,category_2,category_3
0,2021-12-22,Dallas,Property,White Collar,Fraud
1,2021-12-01,Dallas,Property,White Collar,Fraud
2,2021-08-23,Dallas,Property,White Collar,Fraud
3,2021-11-24,Dallas,Property,White Collar,Fraud
4,2021-11-27,Dallas,Property,White Collar,Fraud


# Detroit

In [21]:
city_name = 'Detroit'
file_name = 'detroit'
mapping_columns = ['offense_category']

date_col1 = 'incident_timestamp'
date_format1 = '%Y/%m/%d %H:%M:%S+%f'
# date_col2 = 
# date_format2 = 

In [22]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std['incident_timestamp'] = pd.to_datetime(std['incident_timestamp'], unit='ms')

# Format the datetime column
std['incident_timestamp'] = std['incident_timestamp'].dt.strftime('%Y/%m/%d %H:%M:%S+%f')
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (308, 29)
Length:  1691


,std_date,city,category_1,category_2,category_3
0,2021-08-31,Detroit,Violent,Robbery,Robbery
1,2021-02-01,Detroit,Violent,Robbery,Robbery
2,2021-03-03,Detroit,Violent,Robbery,Robbery
3,2021-04-03,Detroit,Violent,Robbery,Robbery
4,2021-05-09,Detroit,Violent,Robbery,Robbery


# Fort Worth

In [23]:
city_name = 'Fort Worth'
file_name = 'fort_worth'
mapping_columns = ['offense']

date_col1 = 'from_date'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [24]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (39952, 23)
Length:  140658


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Fort Worth,Property,White Collar,Fraud
1,2021-01-01,Fort Worth,Property,White Collar,Fraud
2,2021-01-01,Fort Worth,Property,White Collar,Fraud
3,2021-01-01,Fort Worth,Property,White Collar,Fraud
4,2021-01-01,Fort Worth,Property,White Collar,Fraud


# Los Angeles

In [25]:
city_name = 'Los Angeles'
file_name = 'los_angeles'
mapping_columns = ['crm_cd']

date_col1 = 'date_occ'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [26]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (45691, 34)
Length:  628356


,std_date,city,category_1,category_2,category_3
0,2021-01-02,Los Angeles,Property,Other Property Crime,Other Property Crime
1,2021-01-02,Los Angeles,Property,Other Property Crime,Other Property Crime
2,2021-01-02,Los Angeles,Property,Other Property Crime,Other Property Crime
3,2021-01-02,Los Angeles,Property,Other Property Crime,Other Property Crime
4,2021-01-02,Los Angeles,Property,Other Property Crime,Other Property Crime


# Milwaukee

In [27]:
std = pd.read_csv('raw/raw_incidents_milwaukee.csv', index_col=None, dtype=str)
mapping = pd.read_csv('mappings/mapping_milwaukee.csv', index_col=None, dtype=str)

for col_name in mapping['ColName'].values:
    std[col_name] = std[col_name].astype(str)

def standardize_datetime_milwaukee(d): 
    return datetime.strptime(d, '%Y-%m-%d %H:%M:%S').date()
std['std_date'] = std['ReportedDateTime'].apply(standardize_datetime_milwaukee)
std = std[(std['std_date'] >= start_date) & (std['std_date'] <= end_date)]

all_crime_types = pd.DataFrame()
for col_name in mapping['ColName'].values:
    one_crime_type = std.loc[std[col_name] == '1', ['std_date',col_name]]
    one_crime_type['join_key'] = col_name
    one_crime_type = one_crime_type.drop(col_name, axis=1)
    all_crime_types = pd.concat([all_crime_types, one_crime_type])    
mapping['join_key'] = mapping['ColName'].fillna('').astype(str)
std = pd.merge(all_crime_types, mapping, how='inner', on='join_key')
std['city'] = 'Milwaukee'
std = std[['std_date', 'city', 'category_1', 'category_2', 'category_3']]
std.to_csv('standardized/standardized_incidents_milwaukee.csv', index=False)

print('Length: ', len(std))
std.head()

Length:  124389


,std_date,city,category_1,category_2,category_3
0,2021-01-02,Milwaukee,Violent,Homicide,Murder
1,2021-01-09,Milwaukee,Violent,Homicide,Murder
2,2021-01-09,Milwaukee,Violent,Homicide,Murder
3,2021-01-10,Milwaukee,Violent,Homicide,Murder
4,2021-01-15,Milwaukee,Violent,Homicide,Murder


# New York City

In [28]:
city_name = 'New York City'
file_name = 'new_york_city'
mapping_columns = ['ofns_desc', 'law_cat_cd', 'pd_desc']

date_col1 = 'cmplnt_fr_dt'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [29]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (189160, 44)
Length:  364115


,std_date,city,category_1,category_2,category_3
0,2023-06-03,New York City,Violent,Robbery,Robbery
1,2023-08-05,New York City,Violent,Robbery,Robbery
2,2023-05-17,New York City,Violent,Robbery,Robbery
3,2023-04-15,New York City,Violent,Robbery,Robbery
4,2023-08-31,New York City,Violent,Robbery,Robbery


# Philadelphia

In [30]:
city_name = 'Philadelphia'
file_name = 'philadelphia'
mapping_columns = ['ucr_general', 'text_general_code']

date_col1 = 'dispatch_date'
date_format1 = '%Y-%m-%d'
# date_col2 = 
# date_format2 = 

In [31]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (55297, 23)
Length:  399317


,std_date,city,category_1,category_2,category_3
0,2023-03-11,Philadelphia,Violent,Robbery,Robbery
1,2023-03-11,Philadelphia,Violent,Robbery,Robbery
2,2023-04-12,Philadelphia,Violent,Robbery,Robbery
3,2023-02-22,Philadelphia,Violent,Robbery,Robbery
4,2023-02-22,Philadelphia,Violent,Robbery,Robbery


# Phoenix

In [32]:
city_name = 'Phoenix'
file_name = 'phoenix'
mapping_columns = ['UCR CRIME CATEGORY']

date_col1 = 'OCCURRED ON'
date_format1 = '%Y-%m-%d %H:%M:%S'
date_col2 = 'OCCURRED TO'
date_format2 = '%Y-%m-%d %H:%M:%S'

In [33]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (0, 14)
Length:  182594


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Phoenix,Property,Burglary,Unclassified
1,2021-01-01,Phoenix,Property,Burglary,Unclassified
2,2021-01-01,Phoenix,Property,Burglary,Unclassified
3,2021-01-01,Phoenix,Property,Burglary,Unclassified
4,2021-01-01,Phoenix,Property,Burglary,Unclassified


# San Francisco

In [34]:
city_name = 'San Francisco'
file_name = 'san_francisco'
mapping_columns = ['incident_category']

date_col1 = 'incident_datetime'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [35]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (119428, 33)
Length:  276129


,std_date,city,category_1,category_2,category_3
0,2021-01-01,San Francisco,Property,Theft,Unclassified
1,2021-01-01,San Francisco,Property,Theft,Unclassified
2,2021-01-01,San Francisco,Property,Theft,Unclassified
3,2021-01-01,San Francisco,Property,Theft,Unclassified
4,2021-01-01,San Francisco,Property,Theft,Unclassified


# Seattle

In [36]:
city_name = 'Seattle'
file_name = 'seattle'
mapping_columns = ['offense_parent_group', 'offense']

date_col1 = 'offense_start_datetime'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
date_col2 = 'report_datetime'
date_format2 = '%Y-%m-%dT%H:%M:%S.%f'

In [37]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (8304, 24)
Length:  207461


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Seattle,Property,White Collar,Fraud
1,2021-01-01,Seattle,Property,White Collar,Fraud
2,2021-01-01,Seattle,Property,White Collar,Fraud
3,2021-01-01,Seattle,Property,White Collar,Fraud
4,2021-01-01,Seattle,Property,White Collar,Fraud


# Washington DC

In [38]:
city_name = 'Washington DC'
file_name = 'washington_dc'
mapping_columns = ['OFFENSE']

date_col1 = 'START_DATE'
date_format1 = '%Y/%m/%d %H:%M:%S+%f'
# date_col2 = 
# date_format2 = 

In [39]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std['START_DATE'] = pd.to_datetime(std['START_DATE'], unit='ms')

# Format the datetime column
std['START_DATE'] = std['START_DATE'].dt.strftime('%Y/%m/%d %H:%M:%S+%f')
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (0, 29)
Length:  2985


,std_date,city,category_1,category_2,category_3
0,2021-11-09,Washington DC,Property,Theft,Unclassified
1,2021-05-23,Washington DC,Property,Theft,Unclassified
2,2021-10-17,Washington DC,Property,Theft,Unclassified
3,2021-02-05,Washington DC,Property,Theft,Unclassified
4,2021-10-23,Washington DC,Property,Theft,Unclassified


# Houston

In [72]:
city_name = 'Houston'
file_name = 'houston'
mapping_columns = ['NIBRSDescription']

date_col1 = 'RMSOccurrenceDate'
date_format1 = '%m/%y/%d'
# date_col2 = 
# date_format2 = 

In [60]:
# Houston
df = pd.DataFrame()

# 2021
tmp = pd.read_excel('raw/raw_incidents_houston/NIBRSPublicViewDec21.xlsx')    
df = pd.concat([df, tmp], sort=True)

# 2022
tmp = pd.read_excel('raw/raw_incidents_houston/NIBRSPublicViewDec22.xlsx')    
df = pd.concat([df, tmp], sort=True)

# 2023
tmp = pd.read_excel('raw/raw_incidents_houston/NIBRSPublicViewNov23.xlsx')    
df = pd.concat([df, tmp], sort=True)

In [73]:
std = df
# std['OCCURRED_ON_DATE'] = std['OCCURRED_ON_DATE'].apply(convert_to_datetime_with_offset)
std['RMSOccurrenceDate'] = pd.to_datetime(std['RMSOccurrenceDate'], format='%m/%y/%d')
std['RMSOccurrenceDate'] = std['RMSOccurrenceDate'].dt.strftime('%m/%y/%d')
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping_new(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (276362, 22)
Length:  450931


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Houston,Violent,Assault,Aggravated Assault
1,2021-01-01,Houston,Violent,Assault,Aggravated Assault
2,2021-01-01,Houston,Violent,Assault,Aggravated Assault
3,2021-01-01,Houston,Violent,Assault,Aggravated Assault
4,2021-01-01,Houston,Violent,Assault,Aggravated Assault


# Portland

In [40]:
city_name = 'Portland'
file_name = 'portland'
mapping_columns = ['OffenseType']

date_col1 = 'OccurDate'
date_format1 = '%m/%d/%Y'
# date_col2 = 
# date_format2 = 

In [41]:
# Portland
df = pd.DataFrame()

# 2021
tmp = pd.read_csv('raw/raw_incidents_portland/CrimeData-2021.csv')    
df = pd.concat([df, tmp], sort=True)

# 2022
tmp = pd.read_csv('raw/raw_incidents_portland/CrimeData-2022.csv')    
df = pd.concat([df, tmp], sort=True)

# 2023
tmp = pd.read_csv('raw/raw_incidents_portland/CrimeData-2023.csv')    
df = pd.concat([df, tmp], sort=True)

In [42]:
std = df
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

/var/folders/gx/s91xl6rj6tb3vfb562_hznsh0000gn/T/ipykernel_60017/2212054408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['join_key'] = ''
/var/folders/gx/s91xl6rj6tb3vfb562_hznsh0000gn/T/ipykernel_60017/2212054408.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['join_key'] = df['join_key'] + df[key].fillna('').astype(str)


Dropout: (6532, 20)
Length:  184871


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Portland,Violent,Assault,Simple Assault
1,2021-01-01,Portland,Violent,Assault,Simple Assault
2,2021-01-01,Portland,Violent,Assault,Simple Assault
3,2021-01-02,Portland,Violent,Assault,Simple Assault
4,2021-01-02,Portland,Violent,Assault,Simple Assault


# Denver

In [81]:
city_name = 'Denver'
file_name = 'denver'
mapping_columns = ['offense_type_id']

date_col1 = 'first_occurrence_date'
date_format1 = '%m/%d/%Y %I:%M:%S %p'
date_col2 = 'reported_date'
date_format2 = '%m/%d/%Y %I:%M:%S %p'

In [87]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)
std = standardize_with_mapping_new(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (60310, 26)
Length:  163594


,std_date,city,category_1,category_2,category_3
0,2021-03-08,Denver,Drug,Drug,Drug
1,2022-01-29,Denver,Drug,Drug,Drug
2,2022-08-17,Denver,Drug,Drug,Drug
3,2021-02-11,Denver,Drug,Drug,Drug
4,2022-02-11,Denver,Drug,Drug,Drug


# Nashville

In [45]:
city_name = 'Nashville'
file_name = 'nashville'
mapping_columns = ['offense_description']

date_col1 = 'incident_occurred'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [46]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (144750, 37)
Length:  220086


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Nashville,Violent,Assault,Unclassified
1,2021-01-01,Nashville,Violent,Assault,Unclassified
2,2021-01-01,Nashville,Violent,Assault,Unclassified
3,2021-01-01,Nashville,Violent,Assault,Unclassified
4,2021-01-01,Nashville,Violent,Assault,Unclassified


# Louisville

In [47]:
city_name = 'Louisville'
file_name = 'louisville'
mapping_columns = ['CRIME_TYPE']

date_col1 = 'DATE_OCCURED'
date_format1 = '%Y-%m-%d %H:%M:%S'
date_col2 = 'DATE_REPORTED'
date_format2 = '%Y-%m-%d %H:%M:%S'

In [48]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (304, 36)
Length:  1517


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Louisville,Property,Other Property Crime,Other Property Crime
1,2021-01-01,Louisville,Property,Other Property Crime,Other Property Crime
2,2021-01-01,Louisville,Property,Other Property Crime,Other Property Crime
3,2021-01-02,Louisville,Property,Other Property Crime,Other Property Crime
4,2021-01-01,Louisville,Property,Other Property Crime,Other Property Crime


# Kansas City

In [49]:
city_name = 'Kansas City'
file_name = 'kansas_city'
mapping_columns = ['offense']

date_col1 = 'from_date'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
date_col2 = 'report_date'
date_format2 = '%Y-%m-%dT%H:%M:%S.%f'

In [50]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_two_args(std, date_col1, date_format1, date_col2, date_format2)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (151388, 39)
Length:  149725


,std_date,city,category_1,category_2,category_3
0,2021-08-21,Kansas City,Violent,Homicide,Murder
1,2021-02-17,Kansas City,Violent,Homicide,Murder
2,2021-02-28,Kansas City,Violent,Homicide,Murder
3,2021-01-30,Kansas City,Violent,Homicide,Murder
4,2021-01-10,Kansas City,Violent,Homicide,Murder


# Raleigh

In [51]:
city_name = 'Raleigh'
file_name = 'raleigh'
mapping_columns = ['crime_description']

date_col1 = 'reported_date'
date_format1 = '%Y-%m-%d %H:%M:%S%z'
# date_col2 = 
# date_format2 = 

In [52]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()


Dropout: (61582, 29)
Length:  98105


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Raleigh,Violent,Assault,Simple Assault
1,2021-01-01,Raleigh,Violent,Assault,Simple Assault
2,2021-01-01,Raleigh,Violent,Assault,Simple Assault
3,2021-01-01,Raleigh,Violent,Assault,Simple Assault
4,2021-01-01,Raleigh,Violent,Assault,Simple Assault


# Buffalo

In [53]:
city_name = 'Buffalo'
file_name = 'buffalo'
mapping_columns = ['incident_type_primary']

date_col1 = 'incident_datetime'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [54]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (385, 35)
Length:  36526


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Buffalo,Property,Burglary,Unclassified
1,2021-01-01,Buffalo,Property,Burglary,Unclassified
2,2021-01-01,Buffalo,Property,Burglary,Unclassified
3,2021-01-01,Buffalo,Property,Burglary,Unclassified
4,2021-01-02,Buffalo,Property,Burglary,Unclassified


# Virginia Beach

In [102]:
city_name = 'Virginia Beach'
file_name = 'virginia_beach'
mapping_columns = ['Offense_Description']

date_col1 = 'Date_Occurred'
date_format1 = '%Y-%m-%d %H:%M:%S'
# date_col2 = 
# date_format2 = 

In [103]:
std

,IncidentNumber,Date_Occurred,Date_Found,Offense_Code,Offense_Description,Block,Street,Precinct,Subdivision,Zone_ID,Case_Status,OBJECTID,std_date,join_key
0,2023032921,2023-06-11 03:24:00,1686453840000,13B2,"ASSAULT, SIMPLE, DOMESTIC",700.0,LINDEN CT,4,TIMBERLAKE,426,CLEARED BY ARREST,1,2023-06-11,
1,2023006790,2023-02-03 17:00:00,1675445400000,13C4,THREATEN BODILY HARM,3300.0,HOLLAND RD,1,HOLLAND ELEMENTARY SCHOOL,126,EXCEPTIONALLY CLEARED,2,2023-02-03,
2,2023068794,2023-12-03 15:49:00,1701618600000,13B2,"ASSAULT, SIMPLE, DOMESTIC",4800.0,DEMING CT,4,KEMPSVILLE LAKES,423,CLEARED BY ARREST,3,2023-12-03,
3,2023069081,2023-12-04 21:40:00,1701726600000,13B1,"ASSAULT, SIMPLE",4600.0,LAUDERDALE AVE,3,CHESAPEAKE BEACH,330,PENDING - INACTIVE,4,2023-12-04,
4,2023052104,2023-09-09 08:00:00,1694247120000,13B1,"ASSAULT, SIMPLE",800.0,CLOSE AVE,2,SHADOWLAWN HEIGHTS,225,PENDING - INACTIVE,5,2023-09-09,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2023058684,2023-10-15 12:15:00,1697372100000,VPO,VIOLATE PROTECTIVE ORDER,1100.0,BROADLAWN RD,1,UPTON ESTATES,123,CLEARED BY ARREST,1995,2023-10-15,
1995,2023039437,2023-07-09 22:22:00,1688941320000,13B1,"ASSAULT, SIMPLE",800.0,LYNNHAVEN PKWY,1,LYNNHAVEN AREA,127,PENDING - ACTIVE,1996,2023-07-09,
1996,2023011360,2023-02-26 03:05:00,1677380700000,13B1,"ASSAULT, SIMPLE",4200.0,PLEASANT VALLEY RD,4,ROSEMONT FOREST SOUTH,425,PENDING - INACTIVE,1997,2023-02-26,
1997,2023049370,2023-08-27 02:00:00,1693107480000,13C4,THREATEN BODILY HARM,3200.0,INLET SHORE CT,2,INLET SHORE CONDOS,220,PENDING - ACTIVE,1998,2023-08-27,


In [105]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std['Date_Occurred'] = pd.to_datetime(std['Date_Occurred'], unit='ms')

# Format the datetime column
std['Date_Occurred'] = std['Date_Occurred'].dt.strftime(date_format1)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (284, 18)
Length:  1481


,std_date,city,category_1,category_2,category_3
0,2023-06-11,Virginia Beach,Violent,Assault,Simple Assault
1,2023-12-03,Virginia Beach,Violent,Assault,Simple Assault
2,2023-09-18,Virginia Beach,Violent,Assault,Simple Assault
3,2023-02-09,Virginia Beach,Violent,Assault,Simple Assault
4,2023-03-05,Virginia Beach,Violent,Assault,Simple Assault


# Little Rock

In [90]:
city_name = 'Little Rock'
file_name = 'little_rock'
mapping_columns = ['offense_description']

date_col1 = 'incident_date'
date_format1 = '%Y-%m-%dT%H:%M:%S.%f'
# date_col2 = 
# date_format2 = 

In [91]:
std = pd.read_csv('raw/raw_incidents_' + file_name + '.csv', index_col=None, dtype=str)
std = standardize_datetime_one_arg(std, date_col1, date_format1)
std = standardize_with_mapping(std, city_name, file_name, mapping_columns)
std.head()

Dropout: (0, 20)
Length:  44021


,std_date,city,category_1,category_2,category_3
0,2021-01-01,Little Rock,Violent,Assault,Aggravated Assault
1,2021-01-01,Little Rock,Violent,Assault,Aggravated Assault
2,2021-01-01,Little Rock,Violent,Assault,Aggravated Assault
3,2021-01-01,Little Rock,Violent,Assault,Aggravated Assault
4,2021-01-01,Little Rock,Violent,Assault,Aggravated Assault
